In [1]:
import os
import cv2
import time
import imutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm#_notebook as tqdm
from pyzbar import pyzbar

In [10]:
folderGlob = '/home/rob/HWR/datasets/zooniverse/**/Cropped00[2,4].jpg'
fps = glob(folderGlob)
for fp in tqdm(fps):
    lrfp = fp.replace('Cropped','LowRes')
    img = cv2.imread(fp)
    cv2.imwrite(lrfp,cv2.resize(img,(1024,768)))


  8%|▊         | 303/3833 [00:41<08:04,  7.29it/s]


 16%|█▌        | 605/3833 [01:23<07:34,  7.10it/s]


 24%|██▎       | 907/3833 [02:05<06:37,  7.35it/s]


 31%|███▏      | 1205/3833 [02:47<05:40,  7.71it/s]


 39%|███▉      | 1502/3833 [03:29<05:31,  7.03it/s]


...

 70%|███████   | 2687/3833 [06:15<02:38,  7.23it/s]


 78%|███████▊  | 2983/3833 [06:56<01:57,  7.22it/s]


 86%|████████▌ | 3279/3833 [07:38<01:18,  7.10it/s]


 93%|█████████▎| 3575/3833 [08:20<00:36,  7.15it/s]


100%|██████████| 3833/3833 [08:56<00:00,  7.39it/s]


In [2]:
def rotate_bound(image, angle, borderValue):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH), borderValue=(borderValue,borderValue,borderValue))

In [3]:
y_filter = np.ones((9,9))
y_filter[:4] = -1
y_filter[4] = 0

In [4]:
def sobel(img, dx, dy, ksize):
    sobelx64f = cv2.Sobel(img,cv2.CV_64F,dx,dy,ksize=ksize)
    abs_sobel64f = np.absolute(sobelx64f)
    return np.uint8(abs_sobel64f)

In [5]:
def get_consecutive_coords_above_threshold(sobel_xy, height, width, squ_size, threshold):
    consecutive = 0 
    consecutive_coords = []
    text_coords = []
    for j in range(height//squ_size):
        sy = j*squ_size
        ey = j*squ_size + squ_size
        for i in range(width//squ_size):
            sx = i*squ_size
            ex = i*squ_size + squ_size

            squ = sobel_xy[sy:ey,sx:ex,:]

            _mean = squ.mean()
            if _mean >= threshold:
                consecutive += 1
                consecutive_coords += [(sx,sy,ex,ey)]

            elif consecutive >= 0:
                if consecutive >= min_consecutive:
                    consecutive = 0
                    text_coords += consecutive_coords
                else:
                    consecutive = 0
                    consecutive_coords = []

        if consecutive >= min_consecutive:
            text_coords += consecutive_coords
        consecutive_coords = []
        consecutive = 0
    
    return text_coords

def plot_rectangles(img, rectangles, colors=False):
    if not colors:
        messy = img.copy()
        for (startX, startY, endX, endY) in rectangles:
            cv2.rectangle(messy, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    if colors:
        colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255),(255,255,255),(100,100,100)]
        messy = img.copy()
        color = 0
        for label in rectangles:
            for (startX, startY, endX, endY) in label:
                cv2.rectangle(messy, (startX, startY), (endX, endY), colors[color], 2)
            color += 1
            if color == len(colors):
                color = 0
    plt.figure()
    plt.imshow(messy)
    
def group_squares_into_labels(coords, max_dist_btw, min_squ_per_label):
    coords_stack = coords.copy()

    labels = []
    while len(coords_stack):
        squ1 = coords_stack[0]
        sx1, sy1, ex1, ey1 = squ1
        mx1 = (sx1+ex1)/2
        my1 = (sy1+ey1)/2

        new_label = []
        to_be_checked = [squ1]
        coords_stack.remove(squ1)

        while len(to_be_checked):
            squ1 = to_be_checked[0]
            sx1, sy1, ex1, ey1 = squ1
            mx1 = (sx1+ex1)/2
            my1 = (sy1+ey1)/2

            new_label += [squ1]
            to_be_checked.remove(squ1)

            coords_stack2 = coords_stack.copy()
            for i in range(len(coords_stack2)):
                squ2 = coords_stack2[i]
                sx2, sy2, ex2, ey2 = squ2
                mx2 = (sx2+ex2)/2
                my2 = (sy2+ey2)/2
                dist = np.sqrt((mx1-mx2)**2 + (my1-my2)**2)
                if dist <= max_dist_btw:
                    to_be_checked += [squ2]
                    coords_stack.remove(squ2)
        if len(new_label) >= min_squ_per_label:
            labels += [new_label]
            
    return labels

In [6]:
fp.split('Cropped')[1].split('.jpg')[0]

NameError: name 'fp' is not defined

In [13]:

debug = False

folderGlob = '/home/rob/HWR/datasets/zooniverse/**/LowRes00[2,4].jpg'
fps = glob(folderGlob)
if debug == True:
    fps = np.random.permutation(fps)

labels_data = []
for fp in tqdm(fps):
    if debug == True:
        fp = '/home/rob/HWR/datasets/zooniverse/222344/LowRes002.jpg'
    new_img_size = (480, 640)
    squ_size = 7
    assert squ_size%2 == 1
    max_dist_btw = squ_size*2.4
    min_consecutive = 3
    min_squ_per_label = 40
    threshold = 10
    height, width = new_img_size

    orig = cv2.imread(fp)
    orig_size = orig.shape[:2]
    scale = np.array(orig_size) / np.array(new_img_size)
    img = cv2.resize(orig, (width, height))
    
    img = cv2.GaussianBlur(img, (11,11), 3)
    sobel_xy = sobel(img, 1, 1, 5)    
    
    if debug == True:
        plt.imshow(img)
        plt.figure()
        plt.imshow(sobel_xy)

    text_coords = get_consecutive_coords_above_threshold(sobel_xy, height, width, squ_size, threshold)
    if debug == True:
        plot_rectangles(sobel_xy, text_coords)

    labels = group_squares_into_labels(text_coords, max_dist_btw, min_squ_per_label)
    if debug == True:
        plot_rectangles(img, labels, colors=True)
    
    sobel_y = sobel(img, 0, 1, 3)
    sobel_y = cv2.filter2D(img, -1, y_filter)
    sobel_y = cv2.cvtColor(sobel_y, cv2.COLOR_BGR2GRAY)
    _,sobel_y = cv2.threshold(sobel_y,120,255,cv2.THRESH_BINARY)

    
    #sobel_y = sobel_y / 255
    max_angle = 45
    n_angles = 300
    pad = 5
    
    
    if debug == True:
        plt.figure()
        plt.imshow(img)
        plt.figure()
        plt.imshow(sobel_y)
    i = 0
    catalog_no = fp.split('/')[6]
    for label in labels:
        label_data = [catalog_no, i]
        
        label = np.array(label)
        sx = max(label.min(axis=0)[0]-pad,0)
        sy = max(label.min(axis=0)[1]-pad,0)
        ex = min(label.max(axis=0)[2]+pad,width)
        ey = min(label.max(axis=0)[3]+pad,height)
        
        label_img = sobel_y[sy:ey,sx:ex].copy()
        
        messy = cv2.cvtColor(label_img, cv2.COLOR_GRAY2BGR).copy()
        thetas = []
        thres = 150
        min_len = 80
        max_gap = 20
        lines = cv2.HoughLinesP(label_img,1,np.pi/180,thres,300,min_len, max_gap)
        while (thres > 0) & (lines is None) & (min_len > 0):
            min_len -= 10
            thres -= 10
            max_gap += 10
            lines = cv2.HoughLinesP(label_img,1,np.pi/180,thres,300,min_len, max_gap)
            
        if lines is None:
            lines = []
        
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(messy,(x1,y1),(x2,y2),(0,255,0),2)
            thetas += [np.arctan((y2-y1)/(x2-x1))]
        
        if len(lines):
            best_angle = np.mean(thetas)*180/np.pi
        else:
            best_angle = 0
        
        if debug == True:
            plt.figure()
            plt.imshow(messy)
            plt.title(best_angle)    

        if debug == True:
            sx = int(scale[1]*(sx))
            sy = int(scale[0]*(sy))
            ex = int(scale[1]*(ex))
            ey = int(scale[0]*(ey))
            dst = rotate_bound(orig[sy:ey,sx:ex].copy(), best_angle, borderValue=255)
            plt.figure()
            plt.imshow(dst)
            plt.title(str(best_angle))

        
            

        M = cv2.getRotationMatrix2D((0, 0), best_angle, 1.0)
        rotated_squs = [np.append(a[:2],[1]) for a in label]
        rotated_squs += [np.append(a[2:],[1]) for a in label]
        rotated_squs = np.dot(rotated_squs, M.T)
        sx = rotated_squs.min(axis=0)[0]
        sy = rotated_squs.min(axis=0)[1]
        ex = rotated_squs.max(axis=0)[0]
        ey = rotated_squs.max(axis=0)[1]
        
        sx = int(scale[1]*(sx))
        sy = int(scale[0]*(sy))
        ex = int(scale[1]*(ex))
        ey = int(scale[0]*(ey))
        
        M = cv2.getRotationMatrix2D((0, 0), -best_angle, 1.0)
        corners = [[sx,sy,1],[sx,ey,1],[ex,sy,1],[ex,ey,1]]
        rotated_corners = np.dot(corners, M.T)
        rsx = int(rotated_corners.min(axis=0)[0]) - pad
        rsy = int(rotated_corners.min(axis=0)[1]) - pad
        rex = int(rotated_corners.max(axis=0)[0]) + pad
        rey = int(rotated_corners.max(axis=0)[1]) + pad
        label_data += [rsx, rsy, rex, rey]
        

        x_offset = 0
        y_offset = 0
        if best_angle < 0:
            x_offset = np.abs((orig.shape[0])*np.sin(best_angle*np.pi/180))
        elif best_angle > 0: 
            y_offset = np.abs((orig.shape[1])*np.sin(best_angle*np.pi/180))

        dst = rotate_bound(orig, best_angle, borderValue=255)
            
        nsx = int(sx + x_offset - pad)
        nsy = int(sy + y_offset - pad)
        nex = int(ex + x_offset + pad)
        ney = int(ey + y_offset + pad)
        nsx = max(nsx, 0)
        nsy = max(nsy, 0)
        nex = min(nex, dst.shape[1])
        ney = min(ney, dst.shape[0])
        nsx, nex = min(nsx, nex), max(nsx, nex)
        nsy, ney = min(nsy, ney), max(nsy, ney)
        
        
        aspect_ratio = (nex-nsx)/(ney-nsy)
        if aspect_ratio > 10:
            print('AR too large')
            continue
        
        
        if debug == True:
            plt.figure()
            plt.imshow(dst[nsy:ney,nsx:nex])
        
            plt.figure()
            plt.imshow(rotate_bound(dst[nsy:ney,nsx:nex],-best_angle,borderValue=255))
            plt.title(rotate_bound(dst[nsy:ney,nsx:nex],-best_angle,borderValue=255).shape)
        
        new_fp = fp.replace('Cropped','Label').replace('.jpg','%s.jpg'%i)
        _type = np.nan
        label_data += [new_fp, best_angle]
        if len(pyzbar.decode(dst[nsy:ney,nsx:nex])) | len(pyzbar.decode(cv2.resize(dst[nsy:ney,nsx:nex],(250,100)))):
            _type = 'QR Code'
        label_data += [_type]
        if debug == False:
            cv2.imwrite(new_fp, dst[nsy:ney,nsx:nex])
        labels_data += [label_data]
        i+=1
    if debug == True:
        break




  0%|          | 0/3833 [00:00<?, ?it/s]

  0%|          | 1/3833 [00:02<2:25:53,  2.28s/it]

  0%|          | 2/3833 [00:03<2:03:14,  1.93s/it]

  0%|          | 3/3833 [00:05<2:16:00,  2.13s/it]

  0%|          | 4/3833 [00:08<2:14:07,  2.10s/it]

  0%|          | 5/3833 [00:09<2:01:50,  1.91s/it]

  0%|          | 6/3833 [00:11<2:08:34,  2.02s/it]

  0%|          | 7/3833 [00:15<2:38:23,  2.48s/it]

  0%|          | 8/3833 [00:16<2:17:41,  2.16s/it]

  0%|          | 9/3833 [00:18<2:15:53,  2.13s/it]

  0%|          | 10/3833 [00:20<2:14:02,  2.10s/it]

  0%|          | 11/3833 [00:22<2:02:56,  1.93s/it]

  0%|          | 12/3833 [00:25<2:25:07,  2.28s/it]

  0%|          | 13/3833 [00:27<2:11:37,  2.07s/it]

  0%|          | 14/3833 [00:29<2:12:57,  2.09s/it]

  0%|          | 15/3833 [00:31<2:15:03,  2.12s/it]

AR too large




  0%|          | 16/3833 [00:34<2:24:58,  2.28s/it]

  0%|          | 17/3833 [00:36<2:25:02,  2.28s/it]

  0%|          | 18/3833 [00:38<2:14:35,  2.12s/it]

  0%|          | 19/3833 [00:38<1:40:55,  1.59s/it]

  1%|          | 20/3833 [00:40<1:54:41,  1.80s/it]

  1%|          | 21/3833 [00:42<1:50:15,  1.74s/it]

  1%|          | 22/3833 [00:44<1:53:36,  1.79s/it]

  1%|          | 23/3833 [00:46<2:01:56,  1.92s/it]

  1%|          | 24/3833 [00:48<1:59:41,  1.89s/it]

  1%|          | 25/3833 [00:49<1:54:58,  1.81s/it]

  1%|          | 26/3833 [00:51<1:53:09,  1.78s/it]

  1%|          | 27/3833 [00:54<2:18:20,  2.18s/it]

  1%|          | 28/3833 [00:56<2:15:49,  2.14s/it]

  1%|          | 29/3833 [00:58<2:05:55,  1.99s/it]

  1%|          | 30/3833 [01:02<2:38:34,  2.50s/it]

  1%|          | 31/3833 [01:04<2:32:01,  2.40s/it]

  1%|          | 32/3833 [01:06<2:39:06,  2.51s/it]

  1%|          | 33/3833 [01:10<2:49:16,  2.67s/it]

  1%|          | 34/3833 [01:10<2:05:14,  1.

AR too large




  3%|▎         | 125/3833 [04:46<3:25:26,  3.32s/it]

  3%|▎         | 126/3833 [04:49<3:13:53,  3.14s/it]

  3%|▎         | 127/3833 [04:51<2:52:58,  2.80s/it]

  3%|▎         | 128/3833 [04:53<2:45:28,  2.68s/it]

  3%|▎         | 129/3833 [04:55<2:25:40,  2.36s/it]

  3%|▎         | 130/3833 [04:57<2:14:04,  2.17s/it]

  3%|▎         | 131/3833 [04:59<2:12:45,  2.15s/it]

  3%|▎         | 132/3833 [05:01<2:07:15,  2.06s/it]

  3%|▎         | 133/3833 [05:03<2:04:27,  2.02s/it]

  3%|▎         | 134/3833 [05:06<2:31:59,  2.47s/it]

  4%|▎         | 135/3833 [05:08<2:28:19,  2.41s/it]

  4%|▎         | 136/3833 [05:12<2:42:14,  2.63s/it]

  4%|▎         | 137/3833 [05:14<2:32:34,  2.48s/it]

  4%|▎         | 138/3833 [05:15<2:16:14,  2.21s/it]

  4%|▎         | 139/3833 [05:18<2:21:06,  2.29s/it]

  4%|▎         | 140/3833 [05:20<2:19:20,  2.26s/it]

  4%|▎         | 141/3833 [05:22<2:08:57,  2.10s/it]

  4%|▎         | 142/3833 [05:23<2:02:45,  2.00s/it]

  4%|▎         | 143/3833 

...

AR too large




 88%|████████▊ | 3387/3833 [2:07:33<15:53,  2.14s/it]

 88%|████████▊ | 3388/3833 [2:07:36<16:41,  2.25s/it]

 88%|████████▊ | 3389/3833 [2:07:40<20:28,  2.77s/it]

 88%|████████▊ | 3390/3833 [2:07:45<25:01,  3.39s/it]

 88%|████████▊ | 3391/3833 [2:07:47<23:22,  3.17s/it]

 88%|████████▊ | 3392/3833 [2:07:51<23:41,  3.22s/it]

 89%|████████▊ | 3393/3833 [2:07:57<30:11,  4.12s/it]

 89%|████████▊ | 3394/3833 [2:08:00<28:54,  3.95s/it]

 89%|████████▊ | 3395/3833 [2:08:02<24:19,  3.33s/it]

 89%|████████▊ | 3396/3833 [2:08:04<20:28,  2.81s/it]

 89%|████████▊ | 3397/3833 [2:08:06<19:09,  2.64s/it]

 89%|████████▊ | 3398/3833 [2:08:10<21:44,  3.00s/it]

 89%|████████▊ | 3399/3833 [2:08:12<20:16,  2.80s/it]

 89%|████████▊ | 3400/3833 [2:08:14<17:12,  2.38s/it]

 89%|████████▊ | 3401/3833 [2:08:17<18:05,  2.51s/it]

 89%|████████▉ | 3402/3833 [2:08:18<16:08,  2.25s/it]

 89%|████████▉ | 3403/3833 [2:08:20<15:06,  2.11s/it]

 89%|████████▉ | 3404/3833 [2:08:22<15:28,  2.17s/it]

 89%|███

 95%|█████████▌| 3655/3833 [2:18:13<09:11,  3.10s/it]

 95%|█████████▌| 3656/3833 [2:18:15<07:45,  2.63s/it]

 95%|█████████▌| 3657/3833 [2:18:17<07:15,  2.48s/it]

 95%|█████████▌| 3658/3833 [2:18:19<06:45,  2.32s/it]

 95%|█████████▌| 3659/3833 [2:18:21<06:16,  2.17s/it]

 95%|█████████▌| 3660/3833 [2:18:22<05:57,  2.07s/it]

 96%|█████████▌| 3661/3833 [2:18:24<05:21,  1.87s/it]

 96%|█████████▌| 3662/3833 [2:18:25<04:43,  1.66s/it]

 96%|█████████▌| 3663/3833 [2:18:25<03:42,  1.31s/it]

 96%|█████████▌| 3664/3833 [2:18:28<04:49,  1.71s/it]

 96%|█████████▌| 3665/3833 [2:18:30<04:57,  1.77s/it]

 96%|█████████▌| 3666/3833 [2:18:32<05:07,  1.84s/it]

 96%|█████████▌| 3667/3833 [2:18:34<05:15,  1.90s/it]

 96%|█████████▌| 3668/3833 [2:18:36<04:54,  1.79s/it]

 96%|█████████▌| 3669/3833 [2:18:37<04:45,  1.74s/it]

 96%|█████████▌| 3670/3833 [2:18:39<04:56,  1.82s/it]

 96%|█████████▌| 3671/3833 [2:18:42<05:20,  1.98s/it]

 96%|█████████▌| 3672/3833 [2:18:44<05:29,  2.05s/it]

 96%|█████

AR too large




 96%|█████████▌| 3680/3833 [2:18:59<06:00,  2.36s/it]

 96%|█████████▌| 3681/3833 [2:19:01<05:43,  2.26s/it]

 96%|█████████▌| 3682/3833 [2:19:10<10:22,  4.13s/it]

 96%|█████████▌| 3683/3833 [2:19:12<08:27,  3.38s/it]

 96%|█████████▌| 3684/3833 [2:19:15<08:16,  3.33s/it]

 96%|█████████▌| 3685/3833 [2:19:17<07:30,  3.04s/it]

 96%|█████████▌| 3686/3833 [2:19:19<06:29,  2.65s/it]

 96%|█████████▌| 3687/3833 [2:19:27<10:15,  4.22s/it]

 96%|█████████▌| 3688/3833 [2:19:28<08:06,  3.35s/it]

 96%|█████████▌| 3689/3833 [2:19:30<07:16,  3.03s/it]

 96%|█████████▋| 3690/3833 [2:19:34<08:01,  3.37s/it]

 96%|█████████▋| 3691/3833 [2:19:37<07:27,  3.15s/it]

 96%|█████████▋| 3692/3833 [2:19:39<06:37,  2.82s/it]

 96%|█████████▋| 3693/3833 [2:19:42<06:35,  2.82s/it]

 96%|█████████▋| 3694/3833 [2:19:44<05:59,  2.59s/it]

 96%|█████████▋| 3695/3833 [2:19:46<05:23,  2.34s/it]

 96%|█████████▋| 3696/3833 [2:19:48<05:06,  2.24s/it]

 96%|█████████▋| 3697/3833 [2:19:57<09:49,  4.33s/it]

 96%|███

In [1]:
new_fp

NameError: name 'new_fp' is not defined

In [90]:
dst.shape, nsy, ney, nsx, nex,sy,ey,sx,ex#plt.imshow(dst[nsy:ney,nsx:nex])

((3053, 3157, 3), 1349, 1349, 620, 1322, 1354, 1650, 625, 1317)

In [64]:
       
new_fp = fp.replace('Cropped','Label').replace('.jpg','%s.jpg'%i)
_type = np.nan
label_data += [new_fp, best_angle]
if len(pyzbar.decode(dst[nsy:ney,nsx:nex])) | len(pyzbar.decode(cv2.resize(dst[nsy:ney,nsx:nex],(250,100)))):
    _type = 'QR Code'
label_data += [_type]
if debug == False:
    pass
    #cv2.imwrite(new_fp, dst[nsy:ney,nsx:nex])
labels_data += [label_data]
i+=1

ZeroDivisionError: integer division or modulo by zero

In [14]:
import pandas as pd
df = pd.DataFrame(labels_data, columns=['catalog_no','id','sx','sy','ex','ey','orig_fp','angle', 'type'])
df.to_csv('box coords.csv')

In [93]:
df

,catalog_no,id,sx,sy,ex,ey,orig_fp,angle,type
0,221812,0,210,285,1261,713,/home/rob/HWR/datasets/zooniverse/221812/Label...,0.077486,NaN
1,221812,1,2289,626,2885,1124,/home/rob/HWR/datasets/zooniverse/221812/Label...,4.344374,NaN
2,221812,2,446,822,1948,1214,/home/rob/HWR/datasets/zooniverse/221812/Label...,2.547739,NaN
3,221812,3,2255,1159,3201,1656,/home/rob/HWR/datasets/zooniverse/221812/Label...,-5.173400,NaN
4,221812,4,962,1868,2552,2425,/home/rob/HWR/datasets/zooniverse/221812/Label...,0.105055,QR Code
5,221223,0,1712,1212,2669,1789,/home/rob/HWR/datasets/zooniverse/221223/Label...,-8.947473,NaN
6,221223,1,612,1233,1277,1644,/home/rob/HWR/datasets/zooniverse/221223/Label...,-2.882963,NaN
7,221223,2,677,2039,2251,2635,/home/rob/HWR/datasets/zooniverse/221223/Label...,-0.988999,QR Code
8,222460,0,1687,255,2662,675,/home/rob/HWR/datasets/zooniverse/222460/Label...,1.912178,NaN
9,222460,1,1875,843,3060,1357,/home/rob/HWR/datasets/zooniverse/222460/Label...,-2.965034,NaN


In [47]:
label.shape

(735, 4)